In [4]:
import torch
from craft import CRAFT  # Assuming you have the CRAFT model defined in craft.py
from collections import OrderedDict

In [5]:

# Load the model
craft_net = CRAFT()

# Load the state_dict
state_dict = torch.load('craft_mlt_25k.pth', map_location='cpu')

# Remove the 'module.' prefix if it exists in the state_dict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        new_state_dict[k[7:]] = v  # Remove 'module.' from key
    else:
        new_state_dict[k] = v

# Load the modified state_dict
craft_net.load_state_dict(new_state_dict)

# Set the model to evaluation mode
craft_net.eval()


C:\Users\hamma\AppData\Local\Temp\ipykernel_16044\475445310.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('craft_mlt_25k.pth', map_location='cp

CRAFT(
  (basenet): vgg16_bn(
    (slice1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (slice2): Sequential(
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kerne

In [6]:
import cv2
import numpy as np
from torchvision import transforms

# Load and preprocess the image
image = cv2.imread('test/51+oHGvSvuL.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension


In [7]:
with torch.no_grad():
    prediction = craft_net(input_tensor)

# The output contains two maps: region score and affinity score
region_score = prediction[0][0].cpu().numpy()
affinity_score = prediction[1][0].cpu().numpy()


In [10]:
import craft_utils  # Utility functions for CRAFT (from official repo)
import imgproc      # Image processing utilities (from official repo)

# Post-processing to get bounding boxes
score_text = region_score.copy()
score_link = affinity_score.copy()

# Thresholding
text_threshold = 0.7
link_threshold = 0.4
low_text = 0.4

# Extracting boxes and the score map from the outputs
boxes, polys = craft_utils.getDetBoxes(score_text, score_link, text_threshold, link_threshold, low_text)

# Rescale boxes to the original image size
boxes = craft_utils.adjustResultCoordinates(boxes, image.shape[:2])

# Optional: You can refine polys (polygon) based on boxes here if needed


ValueError: too many values to unpack (expected 2)

In [9]:
# Draw bounding boxes on the image (assuming you have the boxes from post-processing)
for box in boxes:
    cv2.polylines(image, [np.array(box)], isClosed=True, color=(0, 255, 0), thickness=2)

cv2.imshow('Detected Text', image)
cv2.waitKey(0)

NameError: name 'boxes' is not defined